# 상품코드별 월별 예상 금액 산정 분석

이 노트북은 ENTR_BY_INS.csv와 ENTR_INT_INS.csv 데이터를 기반으로 상품코드별 요금제 정보(기본료, 평생할인, 기간할인, 이벤트가, 정책금)를 활용하여 가입 고객의 월별 예상 금액을 산정합니다.

## 분석 목표
- 가입일 기준 M, M+1, M+2... M+n까지의 예상 금액 계산
- 상품코드별 요금제 정책 반영
- ENTR_BY_INS_FORECASTING.csv, ENTR_INT_INS_FORECASTING.csv 파일 생성


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
import matplotlib.font_manager as fm

# 사용 가능한 한글 폰트 찾기
font_list = [f.name for f in fm.fontManager.ttflist if '한글' in f.name or 'Korean' in f.name or 'Malgun' in f.name or 'Nanum' in f.name or 'Noto' in f.name]

if font_list:
    plt.rcParams['font.family'] = font_list[0]
    print(f"✅ 한글 폰트 설정: {font_list[0]}")
else:
    # macOS에서 사용 가능한 한글 폰트들
    mac_fonts = ['AppleGothic', 'Malgun Gothic', 'NanumGothic', 'Noto Sans CJK KR']
    for font in mac_fonts:
        try:
            plt.rcParams['font.family'] = font
            print(f"✅ 한글 폰트 설정: {font}")
            break
        except:
            continue
    else:
        plt.rcParams['font.family'] = 'DejaVu Sans'
        print("⚠️ 한글 폰트를 찾을 수 없어 기본 폰트 사용")

plt.rcParams['axes.unicode_minus'] = False

print("📊 상품코드별 월별 예상 금액 산정 프로그램")
print("=" * 60)


✅ 한글 폰트 설정: Noto Sans Carian
📊 상품코드별 월별 예상 금액 산정 프로그램


In [2]:
# 데이터 로드
print("📁 데이터 로드 중...")

# ENTR_BY_INS.csv 로드 (M-2 정산내역)
try:
    df_entr_by = pd.read_csv('csv/ENTR_BY_INS.csv', encoding='cp949')
    print(f"✅ ENTR_BY_INS.csv 로드 완료: {df_entr_by.shape[0]:,}행 × {df_entr_by.shape[1]}열")
except Exception as e:
    print(f"❌ ENTR_BY_INS.csv 로드 실패: {e}")

# ENTR_INT_INS.csv 로드 (M-1 신규 가입자 정보)
try:
    df_entr_int = pd.read_csv('csv/ENTR_INT_INS.csv', encoding='utf-8')
    print(f"✅ ENTR_INT_INS.csv 로드 완료: {df_entr_int.shape[0]:,}행 × {df_entr_int.shape[1]}열")
except Exception as e:
    print(f"❌ ENTR_INT_INS.csv 로드 실패: {e}")

# MVNO_PRD_PLC.csv 로드 (요금제 정보)
try:
    df_plan = pd.read_csv('csv/MVNO_PRD_PLC.csv', encoding='utf-8')
    print(f"✅ MVNO_PRD_PLC.csv 로드 완료: {df_plan.shape[0]:,}행 × {df_plan.shape[1]}열")
except Exception as e:
    print(f"❌ MVNO_PRD_PLC.csv 로드 실패: {e}")

print(f"\n📋 데이터 로드 완료!")


📁 데이터 로드 중...
✅ ENTR_BY_INS.csv 로드 완료: 270,192행 × 111열
✅ ENTR_INT_INS.csv 로드 완료: 38,161행 × 106열
✅ MVNO_PRD_PLC.csv 로드 완료: 138행 × 9열

📋 데이터 로드 완료!


In [3]:
# 데이터 기본 정보 확인
print("📊 데이터 기본 정보")
print("=" * 50)

print(f"\n🔍 ENTR_BY_INS.csv (M-2 정산내역):")
print(f"  - 총 행 수: {df_entr_by.shape[0]:,}")
print(f"  - 총 열 수: {df_entr_by.shape[1]}")
print(f"  - 메모리 사용량: {df_entr_by.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n🔍 ENTR_INT_INS.csv (M-1 신규 가입자):")
print(f"  - 총 행 수: {df_entr_int.shape[0]:,}")
print(f"  - 총 열 수: {df_entr_int.shape[1]}")
print(f"  - 메모리 사용량: {df_entr_int.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n🔍 MVNO_PRD_PLC.csv (요금제 정보):")
print(f"  - 총 행 수: {df_plan.shape[0]:,}")
print(f"  - 총 열 수: {df_plan.shape[1]}")
print(f"  - 메모리 사용량: {df_plan.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 상품코드 관련 컬럼 확인
print(f"\n📋 상품코드 관련 컬럼:")
entr_by_product_cols = [col for col in df_entr_by.columns if '상품' in col or '코드' in col]
print(f"  - ENTR_BY_INS: {entr_by_product_cols}")

entr_int_product_cols = [col for col in df_entr_int.columns if '상품' in col or '코드' in col or '요금제' in col]
print(f"  - ENTR_INT_INS: {entr_int_product_cols}")

plan_product_cols = [col for col in df_plan.columns if '상품' in col or '코드' in col or '요금제' in col]
print(f"  - MVNO_PRD_PLC: {plan_product_cols}")

# 날짜 관련 컬럼 확인
print(f"\n📅 날짜 관련 컬럼:")
entr_by_date_cols = [col for col in df_entr_by.columns if '일자' in col or '날짜' in col or 'Date' in col]
print(f"  - ENTR_BY_INS: {entr_by_date_cols}")

entr_int_date_cols = [col for col in df_entr_int.columns if '일자' in col or '날짜' in col or 'Date' in col]
print(f"  - ENTR_INT_INS: {entr_int_date_cols}")


📊 데이터 기본 정보

🔍 ENTR_BY_INS.csv (M-2 정산내역):
  - 총 행 수: 270,192
  - 총 열 수: 111
  - 메모리 사용량: 462.30 MB

🔍 ENTR_INT_INS.csv (M-1 신규 가입자):
  - 총 행 수: 38,161
  - 총 열 수: 106
  - 메모리 사용량: 129.72 MB

🔍 MVNO_PRD_PLC.csv (요금제 정보):
  - 총 행 수: 138
  - 총 열 수: 9
  - 메모리 사용량: 0.04 MB

📋 상품코드 관련 컬럼:
  - ENTR_BY_INS: ['MVNO상품코드', 'MVNO상품명', '마켓코드', '가입대리점코드', '실판매점코드', '정산상품코드', '정산상품명', '83.데이터선구매상품명']
  - ENTR_INT_INS: ['유치대리점코드', '최종대리점코드', '실판매POS코드', '실판매POS유형코드', '관리점코드', '상품번호', '서비스코드', '개통요금제코드', '개통요금제명', '현재요금제코드', '현재요금제명', '월말요금제코드', '월말요금제명', '단말기구분코드', '개통모델코드', '현재모델코드', '정책기변코드', '분리형할인코드', '상품개통일', '할부상태코드', '할부청약상태코드', '신용등급코드', '비고1 (단말기임직원추천코드)']
  - MVNO_PRD_PLC: ['요금제코드', '요금제명']

📅 날짜 관련 컬럼:
  - ENTR_BY_INS: ['유효시작일자', '유효종료일자']
  - ENTR_INT_INS: ['처리일자', '기변일자', '취소일자', '해지일자', '최초개통일자']


In [4]:
# 월별 예상 금액 계산 함수 정의
def calculate_monthly_forecast(row, months_ahead=12):
    """
    가입 고객의 월별 예상 금액을 계산하는 함수
    
    Parameters:
    - row: 고객 데이터 행 (pandas Series)
    - months_ahead: 예상할 개월 수 (기본 12개월)
    
    Returns:
    - 월별 예상 금액 리스트
    """
    try:
        # 기본료 (정책금이 있으면 정책금, 없으면 기본료)
        base_fee = row.get('정책금', 0) if pd.notna(row.get('정책금', 0)) and row.get('정책금', 0) > 0 else row.get('기본료', 0)
        
        # 할인 정보
        lifetime_discount = row.get('평생할인', 0) if pd.notna(row.get('평생할인', 0)) else 0
        period_discount = row.get('기간할인', 0) if pd.notna(row.get('기간할인', 0)) else 0
        event_fee = row.get('이벤트가', 0) if pd.notna(row.get('이벤트가', 0)) else 0
        
        # 정책 반영 기간 확인
        policy_start = row.get('정책반영시작일', '1900-01-01')
        policy_end = row.get('정책반영종료일', '9999-12-31')
        
        # 가입일 (ENTR_BY_INS의 경우 처리일자, ENTR_INT_INS의 경우 처리일자)
        join_date = None
        if '처리일자' in row.index and pd.notna(row['처리일자']):
            join_date = pd.to_datetime(row['처리일자'], errors='coerce')
        elif '가입일자' in row.index and pd.notna(row['가입일자']):
            join_date = pd.to_datetime(row['가입일자'], errors='coerce')
        
        if join_date is None or pd.isna(join_date):
            return [0] * months_ahead
        
        # 월별 예상 금액 계산
        monthly_forecasts = []
        
        for month in range(months_ahead):
            # 해당 월의 날짜 계산
            target_date = join_date + relativedelta(months=month)
            
            # 정책 반영 기간 확인
            policy_start_date = pd.to_datetime(policy_start, errors='coerce')
            policy_end_date = pd.to_datetime(policy_end, errors='coerce')
            
            # 정책이 적용되는 기간인지 확인
            is_policy_period = False
            if pd.notna(policy_start_date) and pd.notna(policy_end_date):
                is_policy_period = policy_start_date <= target_date <= policy_end_date
            
            # 월별 금액 계산
            monthly_amount = base_fee
            
            # 평생할인 적용 (정책 기간이거나 평생할인이 있는 경우)
            if lifetime_discount > 0 and (is_policy_period or lifetime_discount > 0):
                monthly_amount = max(0, monthly_amount - lifetime_discount)
            
            # 기간할인 적용 (정책 기간인 경우)
            if period_discount > 0 and is_policy_period:
                monthly_amount = max(0, monthly_amount - period_discount)
            
            # 이벤트가 적용 (정책 기간인 경우)
            if event_fee > 0 and is_policy_period:
                monthly_amount = max(0, monthly_amount - event_fee)
            
            monthly_forecasts.append(monthly_amount)
        
        return monthly_forecasts
    
    except Exception as e:
        print(f"월별 예상 금액 계산 중 오류: {e}")
        return [0] * months_ahead

print("✅ 월별 예상 금액 계산 함수 정의 완료")


✅ 월별 예상 금액 계산 함수 정의 완료


In [5]:
# ENTR_BY_INS 데이터와 요금제 정보 병합
print("🔗 ENTR_BY_INS 데이터와 요금제 정보 병합")
print("=" * 50)

if 'MVNO상품코드' in df_entr_by.columns and '요금제코드' in df_plan.columns:
    # 요금제 정보와 병합
    print("🔄 데이터 병합 중...")
    merged_entr_by = df_entr_by.merge(
        df_plan[['요금제코드', '요금제명', '기본료', '평생할인', '기간할인', '이벤트가', '정책금', '정책반영시작일', '정책반영종료일']], 
        left_on='MVNO상품코드', 
        right_on='요금제코드', 
        how='left'
    )
    
    print(f"✅ ENTR_BY_INS 병합 완료: {merged_entr_by.shape[0]:,}행 × {merged_entr_by.shape[1]}열")
    
    # 매핑 결과 확인
    matched_count = merged_entr_by['요금제코드'].notna().sum()
    unmatched_count = merged_entr_by['요금제코드'].isna().sum()
    total_count = len(merged_entr_by)
    
    print(f"\n📋 매핑 결과:")
    print(f"  - 매핑 성공: {matched_count:,}건 ({matched_count/total_count*100:.1f}%)")
    print(f"  - 매핑 실패: {unmatched_count:,}건 ({unmatched_count/total_count*100:.1f}%)")
    
else:
    print("❌ ENTR_BY_INS와 요금제 정보를 병합할 수 없습니다.")
    print(f"ENTR_BY_INS 컬럼: {[col for col in df_entr_by.columns if '상품' in col or '코드' in col]}")
    print(f"MVNO_PRD_PLC 컬럼: {[col for col in df_plan.columns if '상품' in col or '코드' in col or '요금제' in col]}")
    merged_entr_by = None


🔗 ENTR_BY_INS 데이터와 요금제 정보 병합
🔄 데이터 병합 중...
✅ ENTR_BY_INS 병합 완료: 270,192행 × 120열

📋 매핑 결과:
  - 매핑 성공: 270,192건 (100.0%)
  - 매핑 실패: 0건 (0.0%)


In [6]:
# ENTR_INT_INS 데이터와 요금제 정보 병합
print("\n🔗 ENTR_INT_INS 데이터와 요금제 정보 병합")
print("=" * 50)

if '개통요금제코드' in df_entr_int.columns and '요금제코드' in df_plan.columns:
    # 요금제 정보와 병합
    print("🔄 데이터 병합 중...")
    merged_entr_int = df_entr_int.merge(
        df_plan[['요금제코드', '요금제명', '기본료', '평생할인', '기간할인', '이벤트가', '정책금', '정책반영시작일', '정책반영종료일']], 
        left_on='개통요금제코드', 
        right_on='요금제코드', 
        how='left'
    )
    
    print(f"✅ ENTR_INT_INS 병합 완료: {merged_entr_int.shape[0]:,}행 × {merged_entr_int.shape[1]}열")
    
    # 매핑 결과 확인
    matched_count = merged_entr_int['요금제코드'].notna().sum()
    unmatched_count = merged_entr_int['요금제코드'].isna().sum()
    total_count = len(merged_entr_int)
    
    print(f"\n📋 매핑 결과:")
    print(f"  - 매핑 성공: {matched_count:,}건 ({matched_count/total_count*100:.1f}%)")
    print(f"  - 매핑 실패: {unmatched_count:,}건 ({unmatched_count/total_count*100:.1f}%)")
    
else:
    print("❌ ENTR_INT_INS와 요금제 정보를 병합할 수 없습니다.")
    print(f"ENTR_INT_INS 컬럼: {[col for col in df_entr_int.columns if '상품' in col or '코드' in col or '요금제' in col]}")
    print(f"MVNO_PRD_PLC 컬럼: {[col for col in df_plan.columns if '상품' in col or '코드' in col or '요금제' in col]}")
    merged_entr_int = None



🔗 ENTR_INT_INS 데이터와 요금제 정보 병합
🔄 데이터 병합 중...
✅ ENTR_INT_INS 병합 완료: 38,161행 × 115열

📋 매핑 결과:
  - 매핑 성공: 37,959건 (99.5%)
  - 매핑 실패: 202건 (0.5%)


In [7]:
# ENTR_BY_INS 월별 예상 금액 계산
print("\n📊 ENTR_BY_INS 월별 예상 금액 계산")
print("=" * 50)

if merged_entr_by is not None:
    print("🔄 월별 예상 금액 계산 중...")
    
    # 매핑된 데이터만 사용
    matched_data = merged_entr_by[merged_entr_by['요금제코드'].notna()].copy()
    print(f"  - 계산 대상: {len(matched_data):,}건")
    
    # 월별 예상 금액 계산 (12개월)
    months_ahead = 12
    forecast_columns = [f'M{month+1}' for month in range(months_ahead)]
    
    # 각 고객에 대해 월별 예상 금액 계산
    forecasts = []
    for idx, row in matched_data.iterrows():
        monthly_forecast = calculate_monthly_forecast(row, months_ahead)
        forecasts.append(monthly_forecast)
    
    # 예상 금액을 데이터프레임에 추가
    forecast_df = pd.DataFrame(forecasts, columns=forecast_columns, index=matched_data.index)
    
    # 원본 데이터와 예상 금액 데이터 결합
    entr_by_forecast = matched_data.copy()
    for col in forecast_columns:
        entr_by_forecast[col] = forecast_df[col]
    
    print(f"✅ ENTR_BY_INS 월별 예상 금액 계산 완료")
    print(f"  - 총 고객 수: {len(entr_by_forecast):,}명")
    print(f"  - 예상 기간: {months_ahead}개월")
    
    # 샘플 데이터 확인
    print(f"\n📋 샘플 데이터 (상위 5개):")
    sample_cols = ['가입번호', 'MVNO상품코드', '요금제명', '기본료', '평생할인', '기간할인', '이벤트가', '정책금'] + forecast_columns[:6]
    available_sample_cols = [col for col in sample_cols if col in entr_by_forecast.columns]
    print(entr_by_forecast[available_sample_cols].head().to_string(index=False))
    
    # 월별 예상 금액 통계
    print(f"\n📈 월별 예상 금액 통계:")
    for col in forecast_columns:
        total_amount = entr_by_forecast[col].sum()
        avg_amount = entr_by_forecast[col].mean()
        print(f"  - {col}: 총 {total_amount:,.0f}원, 평균 {avg_amount:,.0f}원")
    
else:
    print("❌ ENTR_BY_INS 병합 데이터가 없습니다.")



📊 ENTR_BY_INS 월별 예상 금액 계산
🔄 월별 예상 금액 계산 중...
  - 계산 대상: 270,192건
✅ ENTR_BY_INS 월별 예상 금액 계산 완료
  - 총 고객 수: 270,192명
  - 예상 기간: 12개월

📋 샘플 데이터 (상위 5개):
        가입번호   MVNO상품코드               요금제명  기본료  평생할인  기간할인  이벤트가  정책금  M1  M2  M3  M4  M5  M6
500224238285 LPZ0002633 [INS]인스 선불정액 300MB    0     0     0     0    0   0   0   0   0   0   0
500224410560 LPZ0002633 [INS]인스 선불정액 300MB    0     0     0     0    0   0   0   0   0   0   0
500224446685 LPZ0002633 [INS]인스 선불정액 300MB    0     0     0     0    0   0   0   0   0   0   0
500224853447 LPZ0002633 [INS]인스 선불정액 300MB    0     0     0     0    0   0   0   0   0   0   0
500224979670 LPZ0002633 [INS]인스 선불정액 300MB    0     0     0     0    0   0   0   0   0   0   0

📈 월별 예상 금액 통계:
  - M1: 총 0원, 평균 0원
  - M2: 총 0원, 평균 0원
  - M3: 총 0원, 평균 0원
  - M4: 총 0원, 평균 0원
  - M5: 총 0원, 평균 0원
  - M6: 총 0원, 평균 0원
  - M7: 총 0원, 평균 0원
  - M8: 총 0원, 평균 0원
  - M9: 총 0원, 평균 0원
  - M10: 총 0원, 평균 0원
  - M11: 총 0원, 평균 0원
  - M12: 총 0원, 평균 0원


In [9]:
# ENTR_INT_INS 월별 예상 금액 계산
print("\n📊 ENTR_INT_INS 월별 예상 금액 계산")
print("=" * 50)

if merged_entr_int is not None:
    print("🔄 월별 예상 금액 계산 중...")
    
    # 매핑된 데이터만 사용
    matched_data = merged_entr_int[merged_entr_int['요금제코드'].notna()].copy()
    print(f"  - 계산 대상: {len(matched_data):,}건")
    
    # 월별 예상 금액 계산 (12개월)
    months_ahead = 12
    forecast_columns = [f'M{month+1}' for month in range(months_ahead)]
    
    # 각 고객에 대해 월별 예상 금액 계산
    forecasts = []
    for idx, row in matched_data.iterrows():
        monthly_forecast = calculate_monthly_forecast(row, months_ahead)
        forecasts.append(monthly_forecast)
    
    # 예상 금액을 데이터프레임에 추가
    forecast_df = pd.DataFrame(forecasts, columns=forecast_columns, index=matched_data.index)
    
    # 원본 데이터와 예상 금액 데이터 결합
    entr_int_forecast = matched_data.copy()
    for col in forecast_columns:
        entr_int_forecast[col] = forecast_df[col]
    
    print(f"✅ ENTR_INT_INS 월별 예상 금액 계산 완료")
    print(f"  - 총 고객 수: {len(entr_int_forecast):,}명")
    print(f"  - 예상 기간: {months_ahead}개월")
    
    # 샘플 데이터 확인
    print(f"\n📋 샘플 데이터 (상위 5개):")
    sample_cols = ['가입번호', '개통요금제코드', '요금제명', '기본료', '평생할인', '기간할인', '이벤트가', '정책금'] + forecast_columns[:6]
    available_sample_cols = [col for col in sample_cols if col in entr_int_forecast.columns]
    print(entr_int_forecast[available_sample_cols].head().to_string(index=False))
    
    # 월별 예상 금액 통계
    print(f"\n📈 월별 예상 금액 통계:")
    for col in forecast_columns:
        total_amount = entr_int_forecast[col].sum()
        avg_amount = entr_int_forecast[col].mean()
        print(f"  - {col}: 총 {total_amount:,.0f}원, 평균 {avg_amount:,.0f}원")
    
else:
    print("❌ ENTR_INT_INS 병합 데이터가 없습니다.")



📊 ENTR_INT_INS 월별 예상 금액 계산
🔄 월별 예상 금액 계산 중...
  - 계산 대상: 37,959건
✅ ENTR_INT_INS 월별 예상 금액 계산 완료
  - 총 고객 수: 37,959명
  - 예상 기간: 12개월

📋 샘플 데이터 (상위 5개):
        가입번호    개통요금제코드                     요금제명  기본료  평생할인  기간할인  이벤트가  정책금  M1  M2  M3  M4  M5  M6
514438614667 LPZ0002642          [INS]인스 LTE선불표준  0.0   0.0   0.0   0.0  0.0 0.0 0.0 0.0 0.0 0.0 0.0
516962662784 LPZ0002642          [INS]인스 LTE선불표준  0.0   0.0   0.0   0.0  0.0 0.0 0.0 0.0 0.0 0.0 0.0
513292045339 LPZ0002633       [INS]인스 선불정액 300MB  0.0   0.0   0.0   0.0  0.0 0.0 0.0 0.0 0.0 0.0 0.0
517165546050 LPZ0002639        [INS]인스 정액선불 11G+  0.0   0.0   0.0   0.0  0.0 0.0 0.0 0.0 0.0 0.0 0.0
515622799160 LPZ0002644 [INS]인스 선불정액 300MB+1Mbps  0.0   0.0   0.0   0.0  0.0 0.0 0.0 0.0 0.0 0.0 0.0

📈 월별 예상 금액 통계:
  - M1: 총 0원, 평균 0원
  - M2: 총 0원, 평균 0원
  - M3: 총 0원, 평균 0원
  - M4: 총 0원, 평균 0원
  - M5: 총 0원, 평균 0원
  - M6: 총 0원, 평균 0원
  - M7: 총 0원, 평균 0원
  - M8: 총 0원, 평균 0원
  - M9: 총 0원, 평균 0원
  - M10: 총 0원, 평균 0원
  - M11: 총 0원, 평균 0원
  - M

In [ ]:
# 월별 예상 금액 시각화
print("\n📊 월별 예상 금액 시각화")
print("=" * 50)

if 'entr_by_forecast' in locals() and 'entr_int_forecast' in locals():
    # 월별 총 예상 금액 계산
    entr_by_monthly_totals = []
    entr_int_monthly_totals = []
    
    for col in forecast_columns:
        entr_by_monthly_totals.append(entr_by_forecast[col].sum())
        entr_int_monthly_totals.append(entr_int_forecast[col].sum())
    
    # 시각화
    plt.figure(figsize=(15, 10))
    
    # 1. 월별 총 예상 금액 비교
    plt.subplot(2, 2, 1)
    months = [f'M{i+1}' for i in range(len(forecast_columns))]
    plt.plot(months, entr_by_monthly_totals, marker='o', label='ENTR_BY_INS', linewidth=2)
    plt.plot(months, entr_int_monthly_totals, marker='s', label='ENTR_INT_INS', linewidth=2)
    plt.title('월별 총 예상 금액 비교', fontsize=14, fontweight='bold')
    plt.xlabel('월')
    plt.ylabel('총 예상 금액 (원)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    # 2. 월별 평균 예상 금액 비교
    plt.subplot(2, 2, 2)
    entr_by_avg = [entr_by_forecast[col].mean() for col in forecast_columns]
    entr_int_avg = [entr_int_forecast[col].mean() for col in forecast_columns]
    plt.plot(months, entr_by_avg, marker='o', label='ENTR_BY_INS', linewidth=2)
    plt.plot(months, entr_int_avg, marker='s', label='ENTR_INT_INS', linewidth=2)
    plt.title('월별 평균 예상 금액 비교', fontsize=14, fontweight='bold')
    plt.xlabel('월')
    plt.ylabel('평균 예상 금액 (원)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    # 3. 요금제별 예상 금액 분포 (ENTR_BY_INS)
    plt.subplot(2, 2, 3)
    if '요금제명' in entr_by_forecast.columns:
        top_plans = entr_by_forecast['요금제명'].value_counts().head(10)
        plan_totals = []
        for plan in top_plans.index:
            plan_data = entr_by_forecast[entr_by_forecast['요금제명'] == plan]
            plan_total = plan_data[forecast_columns].sum().sum()
            plan_totals.append(plan_total)
        
        plt.barh(range(len(top_plans)), plan_totals, color='skyblue', alpha=0.7)
        plt.title('상위 10개 요금제별 총 예상 금액 (ENTR_BY_INS)', fontsize=14, fontweight='bold')
        plt.xlabel('총 예상 금액 (원)')
        plt.yticks(range(len(top_plans)), [plan[:20] + '...' if len(plan) > 20 else plan for plan in top_plans.index])
        plt.grid(True, alpha=0.3)
    
    # 4. 요금제별 예상 금액 분포 (ENTR_INT_INS)
    plt.subplot(2, 2, 4)
    if '요금제명' in entr_int_forecast.columns:
        top_plans = entr_int_forecast['요금제명'].value_counts().head(10)
        plan_totals = []
        for plan in top_plans.index:
            plan_data = entr_int_forecast[entr_int_forecast['요금제명'] == plan]
            plan_total = plan_data[forecast_columns].sum().sum()
            plan_totals.append(plan_total)
        
        plt.barh(range(len(top_plans)), plan_totals, color='lightcoral', alpha=0.7)
        plt.title('상위 10개 요금제별 총 예상 금액 (ENTR_INT_INS)', fontsize=14, fontweight='bold')
        plt.xlabel('총 예상 금액 (원)')
        plt.yticks(range(len(top_plans)), [plan[:20] + '...' if len(plan) > 20 else plan for plan in top_plans.index])
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 시각화 완료!")
    
else:
    print("❌ 예상 금액 데이터가 없습니다.")


In [ ]:
# 결과를 CSV 파일로 저장
print("\n💾 결과 저장")
print("=" * 50)

try:
    # ENTR_BY_INS_FORECASTING.csv 저장
    if 'entr_by_forecast' in locals():
        entr_by_forecast.to_csv('ENTR_BY_INS_FORECASTING.csv', index=False, encoding='utf-8')
        print("✅ ENTR_BY_INS_FORECASTING.csv 저장 완료")
        
        # 저장된 파일 정보
        import os
        file_size = os.path.getsize('ENTR_BY_INS_FORECASTING.csv') / (1024 * 1024)  # MB
        print(f"📁 ENTR_BY_INS_FORECASTING.csv 파일 정보:")
        print(f"  - 크기: {file_size:.2f} MB")
        print(f"  - 행 수: {entr_by_forecast.shape[0]:,}")
        print(f"  - 열 수: {entr_by_forecast.shape[1]}")
    
    # ENTR_INT_INS_FORECASTING.csv 저장
    if 'entr_int_forecast' in locals():
        entr_int_forecast.to_csv('ENTR_INT_INS_FORECASTING.csv', index=False, encoding='utf-8')
        print("✅ ENTR_INT_INS_FORECASTING.csv 저장 완료")
        
        # 저장된 파일 정보
        file_size = os.path.getsize('ENTR_INT_INS_FORECASTING.csv') / (1024 * 1024)  # MB
        print(f"📁 ENTR_INT_INS_FORECASTING.csv 파일 정보:")
        print(f"  - 크기: {file_size:.2f} MB")
        print(f"  - 행 수: {entr_int_forecast.shape[0]:,}")
        print(f"  - 열 수: {entr_int_forecast.shape[1]}")
    
    # 요약 통계 저장
    if 'entr_by_forecast' in locals() and 'entr_int_forecast' in locals():
        summary_data = {
            '구분': ['ENTR_BY_INS', 'ENTR_INT_INS'],
            '총_고객수': [len(entr_by_forecast), len(entr_int_forecast)],
            '12개월_총예상금액': [
                entr_by_forecast[forecast_columns].sum().sum(),
                entr_int_forecast[forecast_columns].sum().sum()
            ],
            '12개월_평균예상금액': [
                entr_by_forecast[forecast_columns].mean().mean(),
                entr_int_forecast[forecast_columns].mean().mean()
            ]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv('FORECASTING_SUMMARY.csv', index=False, encoding='utf-8')
        print("✅ FORECASTING_SUMMARY.csv 저장 완료")
        
        print(f"\n📊 요약 통계:")
        print(summary_df.to_string(index=False))
    
    print(f"\n📁 저장된 파일:")
    if 'entr_by_forecast' in locals():
        print(f"  - ENTR_BY_INS_FORECASTING.csv: ENTR_BY_INS 월별 예상 금액")
    if 'entr_int_forecast' in locals():
        print(f"  - ENTR_INT_INS_FORECASTING.csv: ENTR_INT_INS 월별 예상 금액")
    if 'summary_df' in locals():
        print(f"  - FORECASTING_SUMMARY.csv: 예상 금액 요약 통계")
    
except Exception as e:
    print(f"❌ 파일 저장 중 오류: {e}")

print(f"\n🎉 월별 예상 금액 산정 분석 완료!")


## 📊 분석 결과 요약

이 노트북은 다음과 같은 분석을 수행합니다:

1. **데이터 로드**: ENTR_BY_INS.csv, ENTR_INT_INS.csv, MVNO_PRD_PLC.csv 파일 로드
2. **요금제 정보 병합**: 상품코드와 요금제 정보 연결
3. **월별 예상 금액 계산**: 가입일 기준 M, M+1, M+2... M+12까지의 예상 금액 산정
4. **요금제 정책 반영**: 기본료, 평생할인, 기간할인, 이벤트가, 정책금, 정책반영기간 적용
5. **시각화**: 월별 예상 금액 트렌드 및 요금제별 분포 분석
6. **결과 저장**: 예상 금액 데이터를 CSV 파일로 저장

### 주요 출력 파일:
- `ENTR_BY_INS_FORECASTING.csv`: ENTR_BY_INS 월별 예상 금액 (M1~M12)
- `ENTR_INT_INS_FORECASTING.csv`: ENTR_INT_INS 월별 예상 금액 (M1~M12)
- `FORECASTING_SUMMARY.csv`: 예상 금액 요약 통계

### 🔍 예상 금액 계산 로직:
- **기본료**: 정책금이 있으면 정책금, 없으면 기본료 사용
- **평생할인**: 정책 기간이거나 평생할인이 있는 경우 적용
- **기간할인**: 정책 기간인 경우에만 적용
- **이벤트가**: 정책 기간인 경우에만 적용
- **정책반영기간**: 정책반영시작일~정책반영종료일 범위 내에서만 할인 적용

### 📈 분석 특징:
- **가입일 기준**: 각 고객의 가입일을 기준으로 월별 예상 금액 계산
- **정책 반영**: 요금제별 정책 반영 기간을 고려한 동적 할인 적용
- **12개월 예측**: 가입 후 12개월간의 월별 예상 금액 제공
- **요금제별 분석**: 상품코드별 요금제 정책에 따른 차별화된 예상 금액
